In [7]:
# https://velog.io/@tetrapod0/LLM-%EC%BD%94%EB%93%9C%EC%97%90%EC%84%9C-%EB%8F%8C%EB%A0%A4%EB%B3%B4%EA%B8%B0


from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",    # /v1을 반드시 포함해야 실행됨 (260125)
    api_key="lm-studio",
    model="lmstudio-community/gemma-3-4b-it-GGUF",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "{input} 한국어로 답변해줘."
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke("안녕!")
# response = chain.invoke({'input' : "안녕!"})

안녕하세요! 무엇을 도와드릴까요? 궁금한 점이 있으시거나, 필요한 정보가 있다면 언제든지 말씀해주세요. 😊


In [9]:
from pymongo import MongoClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import json

# --------------------
# 1. MongoDB 연결
# --------------------
client = MongoClient("mongodb://localhost:27017")
db = client["TerryDB"]
collection = db["DailyLog"]

# --------------------
# 2. Aggregation Query
# --------------------
pipeline = [
    {"$unwind": "$exercise"},
    {"$match": {"exercise.subject": "walking"}},
    {
        "$project": {
            "_id": 0,
            "date": 1,
            "steps": "$exercise.steps",
            "weight": "$inbody.weight"
        }
    },
    {"$sort": {"date": 1}}
]

data = list(collection.aggregate(pipeline))
data


[{'date': '2026-01-19', 'steps': 12043, 'weight': 81.1},
 {'date': '2026-01-21', 'steps': 3759, 'weight': 80.4},
 {'date': '2026-01-22', 'steps': 9608, 'weight': 81.2},
 {'date': '2026-01-23', 'steps': 4000, 'weight': 81.4},
 {'date': '2026-01-24', 'steps': 11890, 'weight': 81}]

In [10]:
# --------------------
# 3. LLM에 전달할 입력 문자열 생성
# --------------------
input_text = f"""
다음은 날짜별 걸음 수와 체중 데이터입니다.

{json.dumps(data, ensure_ascii=False, indent=2)}

이 데이터를 기반으로:
1. 걸음 수와 체중 변화의 상관관계를 분석하고
2. 건강 관리 관점에서 인사이트를 도출하고
3. 실천 가능한 행동 제안을 제시해주세요.
"""

# --------------------
# 4. LangChain Prompt 구성
# --------------------
prompt = ChatPromptTemplate.from_template(
    "{input} 한국어로 답변해줘."
)

chain = prompt | llm | StrOutputParser()

# --------------------
# 5. 실행
# --------------------
response = chain.invoke({"input": input_text})
print(response)


## 데이터 분석 및 건강 관리 인사이트 & 행동 제안

제공된 걸음 수와 체중 데이터를 분석한 결과, 다음과 같은 상관관계와 인사이트를 도출할 수 있으며, 이를 바탕으로 실천 가능한 행동 제안을 제시합니다.

**1. 걸음 수와 체중 변화의 상관관계 분석:**

* **전반적인 추세:** 데이터 전체적으로 보면, 걸음 수가 증가하면 체중이 감소하는 경향을 보입니다. 특히 2026-01-19부터 2026-01-24까지 걸음 수에 따라 체중이 점진적으로 감소하는 패턴을 확인할 수 있습니다.
* **구체적인 연관성:**
    * 2026-01-21 (3759걸음, 80.4kg): 걸음 수가 줄어들면서 체중도 약간 감소했습니다.
    * 2026-01-22 (9608걸음, 81.2kg): 걸음 수가 다시 증가하면서 체중이 소폭 증가했습니다.
    * 2026-01-23 (4000걸음, 81.4kg): 걸음 수 변화 없이 체중이 약간 증가했습니다.
    * 2026-01-24 (11890걸음, 81kg): 가장 높은 걸음 수를 기록하며 체중이 감소했습니다.

* **상관관계의 한계:**  단순히 걸음 수와 체중의 상관관계만으로는 인과 관계를 확정할 수 없습니다. 다른 요인 (식습관, 수면, 스트레스 등) 또한 체중에 영향을 미칠 수 있기 때문입니다.


**2. 건강 관리 관점에서 인사이트 도출:**

* **활동량 증가의 긍정적 효과:** 걸음 수가 늘어날수록 체중 감소에 긍정적인 영향을 미치는 것으로 보아, 꾸준한 활동량 증진이 체중 관리에 도움이 될 수 있다는 점을 시사합니다.
* **최고 걸음 수 시 체중 감소:**  11890걸음을 기록했을 때 체중이 가장 크게 감소한 것은, 특정 수준 이상의 활동량이 체중 감량에 더욱 효과적일 수 있음을 보여줍니다. (단, 이는 데이터셋의 크기가 작기 때문에 일반화하기는 어렵습니다.)
* **체중 변화의 변동성:**  데이터에서 볼 수 있는 체중 변화의 변동성은 건강 관리를 위한 꾸준함의 중요성을 강조합니다. 일시적